In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
import torch  
from torch.utils.data import DataLoader

from curious.utils.utils import move_paddings_to_right
from curious.config import TrainingConfig, RLConfig, WandbConfig, BaseConfig, SamplingConfig, RewardConfig, SFLConfig 
from curious.train.training_setup import set_up_training
from curious.train.trainer import PolicyGradientTrainer
from curious.sampling.sampling import sequences_log_probs 
from curious.policy_gradient.loss import ActorLoss
from curious.replay.experience import Experience, ReplayBuffer, join_experience_batch

PAD_TOKEN_ID = 128

/home/zeus/miniconda3/envs/cloudspace/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-05-11 09:47:53,585] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/system/conda/miniconda3/envs/cloudspace/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/system/conda/miniconda3/envs/cloudspace/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/system/conda/miniconda3/envs/cloudspace/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/system/conda/miniconda3/envs/cloudspace/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/system/conda/miniconda3/envs/cloudspace/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `d

# Utils 
*** 

In [2]:
attention_mask = torch.tensor(
    [
        [0, 0, 1, 1, 1],
        [0, 1, 1, 1, 1],
    ]
)

input_ids = torch.tensor(
    [
        [PAD_TOKEN_ID, PAD_TOKEN_ID, 1, 2, 3,],
        [PAD_TOKEN_ID, 1,            2, 3, 4,],
    ]
)


sequence_ids = torch.tensor(
    [
        [PAD_TOKEN_ID, PAD_TOKEN_ID, 1, 2, 3, 890, 891, PAD_TOKEN_ID, PAD_TOKEN_ID, PAD_TOKEN_ID],
        [PAD_TOKEN_ID, 1,            2, 3, 4, 899, 900, 911,          PAD_TOKEN_ID, PAD_TOKEN_ID],
    ]
)

move_paddings_to_right(input_ids, attention_mask, sequence_ids, PAD_TOKEN_ID)

(tensor([[  1,   2,   3, 890, 891, 128, 128],
         [  1,   2,   3,   4, 899, 900, 911]]),
 tensor([[False, False,  True,  True, False, False],
         [False, False, False,  True,  True,  True]]))

# Loading 
*** 

In [3]:
train_config = TrainingConfig(
    rl_config=RLConfig(
        group_size=3, 
        mini_batch_size=6,
        kl_weight=0,
        
    ),
    wandb_config=WandbConfig(),
    base_config=BaseConfig(
        train_batch_size=2,
        
    ),
    sampling_config=SamplingConfig(
        max_new_tokens=256,
    ),
    reward_config=RewardConfig(),
    sfl_config=SFLConfig(
    ),
)
#train_config.rl_config
training_setup, init_train_state = set_up_training(train_config)

#### Loading target policy Qwen/Qwen2-0.5B-Instruct ####
Applied Liger kernels to Qwen2
#### Loading dataset openai/gsm8k ####
Detected train_max_length: 212
Setting train_max_length to 212
Detected test_max_length: 188
Setting test_max_length to 188
#### Loading rollout data loader ####
#### Defining actor loss ####
#### Defining reward model ####
#### Defining generation config ####
#### Defining evaluation config ####
#### Defining optimizer ####
#### Defining lr scheduler ####


In [4]:
init_train_state.keys()

dict_keys(['run_name', 'device', 'seed', 'model', 'optimizer', 'lr_scheduler', 'reference_model', 'kl_controller'])

In [5]:
trainer = PolicyGradientTrainer(training_setup) 

In [6]:
batch_inputs = next(iter(trainer.rollout_data_loader))

In [7]:
replay_buffer = trainer.collect_trajectories(
    init_train_state, 
    batch_inputs, 
    0 
)

W0511 09:48:09.775000 47207 /system/conda/miniconda3/envs/cloudspace/lib/python3.11/site-packages/torch/_dynamo/variables/tensor.py:869] [0/0] Graph break from `Tensor.item()`, consider setting:
W0511 09:48:09.775000 47207 /system/conda/miniconda3/envs/cloudspace/lib/python3.11/site-packages/torch/_dynamo/variables/tensor.py:869] [0/0]     torch._dynamo.config.capture_scalar_outputs = True
W0511 09:48:09.775000 47207 /system/conda/miniconda3/envs/cloudspace/lib/python3.11/site-packages/torch/_dynamo/variables/tensor.py:869] [0/0] or:
W0511 09:48:09.775000 47207 /system/conda/miniconda3/envs/cloudspace/lib/python3.11/site-packages/torch/_dynamo/variables/tensor.py:869] [0/0]     env TORCHDYNAMO_CAPTURE_SCALAR_OUTPUTS=1
W0511 09:48:09.775000 47207 /system/conda/miniconda3/envs/cloudspace/lib/python3.11/site-packages/torch/_dynamo/variables/tensor.py:869] [0/0] to include these operations in the captured graph.
W0511 09:48:09.775000 47207 /system/conda/miniconda3/envs/cloudspace/lib/pytho

Batch indx 0

torch.cuda.memory_allocated: 0.928201GB

torch.cuda.memory_reserved: 1.644531GB

torch.cuda.max_memory_reserved: 1.681641GB

{
    'train/mean_batch_returns': 0.0,
    'train/mean_batch_solved_rate': 0.5,
    'train/mean_num_words_in_completions': 14.5,
    'train/max_num_words_in_completions': 32.0,
    'train/min_num_words_in_completions': 1.0,
    'train/mean_batch_format_returns': 0.0,
    'train/mean_batch_outcome_returns': 0.0,
    'train/mean_batch_length_penalty': 0.0,
    'train/lr': 1e-06,
    'train/mean_action_entropy': 0.2890625,
    'num_batches_visited': 0
}

In [8]:
exp = replay_buffer[0]

In [9]:
tokenizer = trainer.tokenizer

In [10]:
for key in exp.keys:
    val = getattr(exp, key)
    if val is not None:
        print(key, '->', val.shape)

sequences -> torch.Size([270])
action_log_probs -> torch.Size([269])
returns -> torch.Size([])
solved_mask -> torch.Size([])
advantages -> torch.Size([])
attention_mask -> torch.Size([270])
action_mask -> torch.Size([269])


In [11]:
print(tokenizer.decode(exp.sequences))

<|im_start|>system
Please reason step by step, and put your final answer within $\boxed{}$.<|im_end|>
<|im_start|>user
Janessa has a plan to give her brother Dexter his first collection of baseball cards. She currently has 4 cards in addition to the 13 that her father gave her.  She ordered a collection of 36 cards from eBay. After inspecting the cards she found 4 cards are in bad shape and decides to throw them away. Janessa ended up giving Dexter 29 cards. How many cards did Janessa keep for herself?<|im_end|>
<|im_start|>assistant
 Janessa initially has 4 cards and her father gave her an additional 13 cards, so she now has a total of 4 + 13 = 17 cards.
She then ordered a collection of 36 cards from eBay, so she now has a total of 17 + 36 = 53 cards.
After inspecting the cards, she found 4 cards in bad shape, so she had 53 - 4 = 49 cards remaining.
After giving Dexter 29 cards, she ended up giving him 29 cards, so she kept 49 - 29 = 20 cards for herself.
The answer is: $\boxed{20}$.<

In [12]:
print(
    tokenizer.decode(
        exp.sequences[exp.attention_mask]
    )
)

<|im_start|>system
Please reason step by step, and put your final answer within $\boxed{}$.<|im_end|>
<|im_start|>user
Janessa has a plan to give her brother Dexter his first collection of baseball cards. She currently has 4 cards in addition to the 13 that her father gave her.  She ordered a collection of 36 cards from eBay. After inspecting the cards she found 4 cards are in bad shape and decides to throw them away. Janessa ended up giving Dexter 29 cards. How many cards did Janessa keep for herself?<|im_end|>
<|im_start|>assistant
 Janessa initially has 4 cards and her father gave her an additional 13 cards, so she now has a total of 4 + 13 = 17 cards.
She then ordered a collection of 36 cards from eBay, so she now has a total of 17 + 36 = 53 cards.
After inspecting the cards, she found 4 cards in bad shape, so she had 53 - 4 = 49 cards remaining.
After giving Dexter 29 cards, she ended up giving him 29 cards, so she kept 49 - 29 = 20 cards for herself.
The answer is: $\boxed{20}$.<

In [13]:
print(exp.returns)

tensor(1., dtype=torch.bfloat16)


In [14]:
print(exp.solved_mask)

tensor(1., dtype=torch.bfloat16)


In [15]:
print(exp.advantages)

tensor(0., dtype=torch.bfloat16)


In [16]:
exp1 = replay_buffer[1]
print(exp1.solved_mask)
print(exp1.returns)
print(exp1.advantages)
print(
    tokenizer.decode(
        exp1.sequences[exp1.attention_mask]
    )
)

tensor(1., dtype=torch.bfloat16)
tensor(1., dtype=torch.bfloat16)
tensor(0., dtype=torch.bfloat16)
<|im_start|>system
Please reason step by step, and put your final answer within $\boxed{}$.<|im_end|>
<|im_start|>user
Janessa has a plan to give her brother Dexter his first collection of baseball cards. She currently has 4 cards in addition to the 13 that her father gave her.  She ordered a collection of 36 cards from eBay. After inspecting the cards she found 4 cards are in bad shape and decides to throw them away. Janessa ended up giving Dexter 29 cards. How many cards did Janessa keep for herself?<|im_end|>
<|im_start|>assistant
 Janessa initially has 4 cards and her father gave her 13 cards, so she now has 4 + 13 = 17 cards.
She then ordered a collection of 36 cards from eBay, so she now has 17 + 36 = 53 cards.
After inspecting the cards, she found 4 cards in bad shape and decides to throw them away, so she now has 53 - 4 = 49 cards.
She ended up giving Dexter 29 cards, so she kept 

In [17]:
exp2 = replay_buffer[2]
print(exp2.solved_mask)
print(exp2.returns)
print(exp2.advantages)
print(
    tokenizer.decode(
        exp2.sequences[exp2.attention_mask]
    )
)

tensor(1., dtype=torch.bfloat16)
tensor(1., dtype=torch.bfloat16)
tensor(0., dtype=torch.bfloat16)
<|im_start|>system
Please reason step by step, and put your final answer within $\boxed{}$.<|im_end|>
<|im_start|>user
Janessa has a plan to give her brother Dexter his first collection of baseball cards. She currently has 4 cards in addition to the 13 that her father gave her.  She ordered a collection of 36 cards from eBay. After inspecting the cards she found 4 cards are in bad shape and decides to throw them away. Janessa ended up giving Dexter 29 cards. How many cards did Janessa keep for herself?<|im_end|>
<|im_start|>assistant
 Janessa initially has 4 cards and her father gave her 13, so she has a total of 4 + 13 = 17 cards.
She then ordered a collection of 36 cards from eBay, so she now has a total of 17 + 36 = 53 cards.
After inspecting the cards, she found 4 cards in bad shape and decides to throw them away, so she now has 53 - 4 = 49 cards.
Janessa ended up giving Dexter 29 car

In [18]:
exp3 = replay_buffer[3]
print(exp3.solved_mask)
print(exp3.returns)
print(exp3.advantages)
print(
    tokenizer.decode(
        exp3.sequences[exp3.attention_mask]
    )
)

tensor(0., dtype=torch.bfloat16)
tensor(-1., dtype=torch.bfloat16)
tensor(0., dtype=torch.bfloat16)
<|im_start|>system
Please reason step by step, and put your final answer within $\boxed{}$.<|im_end|>
<|im_start|>user
Quentavious has 5 nickels. His friend offers him some gum and will give him two pieces per nickel. If Quentavious leaves with 2 nickels, how many pieces of gum did he get?<|im_end|>
<|im_start|>assistant
 Quentavious starts with 5 nickels, but he leaves with 2 nickels, so he has 5 - 2 = 3 nickels left.
His friend gives him 2 pieces of gum per nickel, so he gets 3 x 2 = 6 pieces of gum.
The answer is: $\boxed{6}$<|im_end|>


In [19]:
print(
    tokenizer.decode(
        exp3.sequences[1:][exp3.action_mask]
    )
)

 Quentavious starts with 5 nickels, but he leaves with 2 nickels, so he has 5 - 2 = 3 nickels left.
His friend gives him 2 pieces of gum per nickel, so he gets 3 x 2 = 6 pieces of gum.
The answer is: $\boxed{6}$<|im_end|>


In [20]:
experience_sampler = DataLoader(
    replay_buffer,
    batch_size=trainer.rl_config.mini_batch_size,
    shuffle= False,
    drop_last=False,
    collate_fn=join_experience_batch,
    num_workers=trainer.base_config.num_workers,
)

In [21]:
next_exp = next(iter(experience_sampler))

In [22]:
next_exp.returns.shape, next_exp.advantages.shape 

(torch.Size([6]), torch.Size([6]))

In [23]:
for key in next_exp.keys:
    val = getattr(next_exp, key)
    if val is not None:
        print(key, '->', val.shape)

sequences -> torch.Size([6, 270])
action_log_probs -> torch.Size([6, 269])
returns -> torch.Size([6])
solved_mask -> torch.Size([6])
advantages -> torch.Size([6])
attention_mask -> torch.Size([6, 270])
action_mask -> torch.Size([6, 269])


In [24]:
next_exp = next_exp.to(init_train_state["model"].device) 
log_probs, _ = sequences_log_probs(
    init_train_state["model"], 
    sequence_ids= next_exp.sequences, 
    attention_mask= next_exp.attention_mask,
    return_entropy=False,
    logits_minibatch_size=trainer.rl_config.logits_minibatch_size,
)

In [25]:
print(log_probs.shape)

torch.Size([6, 269])


In [26]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

loss, mean_kl, mean_actor_loss = trainer.actor_loss(
    log_probs=log_probs, 
    experience= next_exp
)

In [27]:
print(loss)
print(mean_kl)
print(mean_actor_loss)

tensor(0., device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<CompiledFunctionBackward>)
0
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<CompiledFunctionBackward>)
